## Association Request

This notebook aims to give a brief explanation on  how the association request works.

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import syft as sy
import torch as th

In [2]:
sy.networks

,name,host_or_ip,vpn_host_or_ip,protocol,port,admin_email,website,slack,slack_channel
0,omnet,13.64.187.229,100.64.0.1,http,80,support@openmined.org,https://www.openmined.org/,https://slack.openmined.org/,omnet
1,United Nations PET Lab (Mock),168.61.66.213,100.64.0.1,http,80,support@openmined.org,https://www.openmined.org/,https://slack.openmined.org/,petlab


# start two different stacks
```
hagrid launch test_network_1 network to docker:9081
hagrid launch test_domain_1 domain to docker:9082
```

### Connect with Domain

In [ ]:
NETWORK_HOST_IP = "localhost:9081"

In [ ]:
domain = sy.login(email="info@openmined.org", password="changethis", port=9082)

### Request Node Association

In [ ]:
vpn_status = domain.vpn_status()

In [ ]:
vpn_status

In [ ]:
DOMAIN_VPN_IP = vpn_status["host"]["ip"]
for peer in vpn_status["peers"]:
    if peer["hostname"] == "test_network_1":
        NETWORK_VPN_IP = peer["ip"]

In [ ]:
print(DOMAIN_VPN_IP, NETWORK_VPN_IP)

In [ ]:
# we need to refactor this so that the VPN IPs are not required to be supplied
domain.apply_to_network(
    domain_vpn_ip=DOMAIN_VPN_IP,
    network_vpn_ip=NETWORK_VPN_IP
)

### Monitor Association Requests

In [ ]:
network = sy.login(email="info@openmined.org", password="changethis", port=9081)

In [ ]:
network.association.pandas()

### Accept/Deny Association Requests

In [ ]:
associations = network.association.all()

In [ ]:
request_id = int(associations[0]["association_id"])

In [ ]:
network.association[request_id].accept()

### Create Data and Tag

In [ ]:
import torch as th
x = th.Tensor([1,2,3])

In [ ]:
x_ptr = x.send(domain, tags=["secret"])

In [ ]:
domain.store

### Use Domain's Proxy Client

In [10]:
import syft as sy
import torch as th

In [ ]:
NETWORK_HOST_IP="13.64.187.229"
network = sy.login(url=f"http://{NETWORK_HOST_IP}", port=80)

In [6]:
network.domains

,host_or_ip,id,is_vpn,name
0,100.64.0.2,3d405798bd374911900793e382db5242,1,outrageous_oneill


In [7]:
# you can use 3 different ways to get the proxy domain client

# with int row
domain_client = network.domains[0]

In [8]:
domain_client

<ProxyClient - outrageous_oneill: <UID: 3d405798bd374911900793e382db5242>>

In [ ]:
# with id string
domain_client = network.domains["9a7fd94214e049f6871d3d21e274cd29"]

In [ ]:
# with UID object
from syft.core.common.uid import UID
id = UID("9a7fd94214e049f6871d3d21e274cd29")
domain_client = network.domains[id]

In [ ]:
# now we can actually use it

In [ ]:
domain_client

In [ ]:
domain_client.store

In [ ]:
secret_ptr = domain_client.store[0]

In [ ]:
try:
    res = secret_ptr.get(delete_obj=False)
    print("Now I can get", res)
except Exception as e:
    print("Guest cant get")

### Login to Remote Domain via Proxy Client

In [ ]:
domain_client.login(email="info@openmined.org", password="changethis")

In [ ]:
domain_client.logged_in

In [ ]:
try:
    res = secret_ptr.get(delete_obj=False)
    print("Now I can get", res)
except Exception as e:
    print("Guest cant get")

In [ ]:
domain_client.logout()

### Search the Network

In [1]:
import syft as sy

In [2]:
sy.networks

In [3]:
NETWORK_HOST_IP="13.64.187.229"
network = sy.login(url=f"http://{NETWORK_HOST_IP}", port=80)

Connecting to http://13.64.187.229...

No email and password defined in login() - connecting as anonymous user!!!

 done! 	 Logging into omnet... done!


In [4]:
network.domains

,host_or_ip,id,is_vpn,name
0,100.64.0.2,3d405798bd374911900793e382db5242,1,outrageous_oneill


In [5]:
network.search(query=["secret"])

,id,name,host_or_ip,is_vpn
0,3d405798bd374911900793e382db5242,outrageous_oneill,100.64.0.2,True


In [10]:
oniel_guest = network.domains['3d405798bd374911900793e382db5242']

In [17]:
oniel_root = oniel_guest.login(email="info@openmined.org", password="changethis")

Logged in to outrageous_oneill as info@openmined.org


In [18]:
ptr = oniel_root.store['secret']

In [19]:
ptr.get()

tensor([1., 2., 3.])